# Funcional

# Code Assigment 1

For this assignment you will use the following SVM implementation for classifying these datasets:

https://archive.ics.uci.edu/ml/datasets/banknote+authentication


https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+

You should:

1) Specify which Machine Learning problem are you solving.

2) Provide a short summary of the features and the labels you are working on.

3) Please answer the following questions: a) Are these datasets linearly separable? b) Are these datasets randomly chosen and c) The sample size is enough to guarantee generalization.

4) Provide an explanation how and why the code is working. You can add comments and/or formal explanations into the notebook.

5) Show some examples to illustrate that the method is working properly.

6) Provide quantitative evidence for generalization using the provided dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from models import  train_validation_test_split,SVM
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
np.random.seed(42)


import matplotlib.pyplot as plt
import seaborn as sns

# Bank note model 

1) Specify which Machine Learning problem are you solving.

2) Provide a short summary of the features and the labels you are working on.

3. Please answer the following questions: 
    1. Are these datasets linearly separable? 
    2. Are these datasets randomly chosen.
    3. The sample size is enough to guarantee generalization.

4) Provide an explanation how and why the code is working. You can add comments and/or formal explanations into the notebook.

5) Show some examples to illustrate that the method is working properly.

6) Provide quantitative evidence for generalization using the provided dataset.

Por hacer 

* Investigar más sobre Curtosis 
* Punto 4
* Punto 5

Segundo dataset

* Punto 1
* Punto 2
* Punto 3
* Punto 4
* Punto 5 

## Solution

### 1 Specify which Machine Learning problem are you solving.
The machine learning problem in this case is to detect counterfeit banknotes, the data is obtained from https://archive.ics.uci.edu/ml/datasets/banknote+authentication , in this case we are in front of a classification problem,  where we try to classify each banknote into two possible classes, counterfeits and genuine banknotes, the objective is to maximize the detection of counterfeit banknotes.

In [ ]:
# Load the dataset
df_bank_note = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt', header=None)

df_bank_note.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'class']

In [ ]:
df_bank_note['class']=df_bank_note['class'].replace(0,-1)

In [ ]:
df_bank_note

In [ ]:
df_bank_note=shuffle(df_bank_note)

In [ ]:
df_bank_note['class']

In based on this data,in effect, variance, skewness, curtosis and entropy we want to determine the class of every banknote.

In [ ]:
sum(df_bank_note['class']==0)

In [ ]:
sum(df_bank_note['class']==1)

### 2  Provide a short summary of the features and the labels you are working on.

For create this dataset was required to extract the features from images of genuine and forged banknote-like specimens, to do this they use wavelet transformation tool.we are going to introduce briefly the wavelet transformation tool,this transformations works similar to a Fourier tranformation, with the difference that the functions used are wavelets, that is functions of the form:

$$ \psi = -(x-b)e^{\frac{-(x-b)^{2}/2a²}{\sqrt{2\pi}a^{3}}} $$

Were $a$ determines scale and $b$ determines location,the wavelets have the advantage of extract local information , that is, if we want a better model for local information, wavelet transform work better than Fourier tranform, in this case since the genuine banknotes have local information to proof that is in fact genuine (like watermarks and othes markers) this information is so relevant, that is the reason for use wavelet transformation insted of Fourier or other transform,the wavelet transform in the  from this transformation the features want to describe the wavelets distributions,following this reasoning we obtain.

1. variance of Wavelet Transformed image (continuous):
    Over the distribution of wavelets transformation, we want to know his variance, the variance is a interesting value for our problem because a forged banknote probably have diferent ranges that a genuine.
2. Skewness of Wavelet Transformed image (continuous):
    The skewness give us a measure of the simetry of the distribution, we can think that exists a correlation between a class and his simetry of distribution.
3. Curtosis of Wavelet Transformed image (continuous)
    Curtosis is a measure of the density of the distribution,that give us information about the distribution of the pixels and can describe a principal element of the banknote.
    
4. Entropy of image (continuous)
    If we work with pixels like a random variable, the entropy show us the "disorder" or uncertainty in this random variable, the use of specific materials and inks determine if a banknote are genuine or not, this can be characterised with the entropy, because the behaviour of the pixels with some materials can be defined by his entropy.  
5. class (integer)
    In this case we have two categories, the dataset description do not have the meaning of the class, but we could think that 0 is genuine and 1 is forged.

In [ ]:
df_bank_note.describe()

Here we can observe the different statistics measures of the data as a summary of principal statistical information of every variable.

### 3 Answer questions

* ¿Are this dataset linearly separable?

To see that this dataset are linearly separable we have to find a separator hiperplane who induce two partitions, in both cases we only need to find the support vector machine, if we can find it, then the data are linearly separable. But we could also run a clustering algorithm to see that we could separate the set of points in two collections, we are going to run a logistic regression algorithm to see that the data are separable and after a SVM algorithm.Also we are going to visualize the dataset and see if the two datasets looks like if were separables,in fact:

In [ ]:
# Visualizing data

pair=sns.pairplot(df_bank_note, hue="class")
pair.fig.subplots_adjust(top=.93)
pair.fig.suptitle('Marginal plot and distributions')

We can see that the distributions of de data in both cases looks different and distinguible one for another.And also we can see a correlation matrix to see that the data are related to the labels.

In [ ]:
sns.set_theme(style="dark")
corr = df_bank_note.corr()
mask = np.triu(np.ones_like(corr, dtype=bool)).T
f, ax = plt.subplots(figsize=(6,4))
ax.set_title("Correlation Matrix")
# Generate a custom diverging colormap
# cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, center=0,cmap='coolwarm',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
X =df_bank_note.drop('class',axis=1).values
y= df_bank_note['class'].values

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_val, X_test, y_train, y_val, y_test= train_validation_test_split(X, y)

Now we are going to see if the data are linearly separable with a logistic regression over all the data

In [ ]:
clf = LogisticRegression(tol=0,max_iter=10000)
clf.fit(X,y)

In [ ]:
clf.score(X,y)

With this information we now see that a model can separate with an low error the dataset in to classes. And with this we can say that are linearly separable.

* ¿Are this dataset randomly chosen?

The function  *train_validation_test_split* is designed to randomly choose the data and split in 3 sets, i.e, train, test and validation, we can see that the distribution of the data is preserved, to do this we plot the train data.

In [ ]:
pair=sns.pairplot(pd.DataFrame(X_train))
pair.fig.subplots_adjust(top=.93)
pair.fig.suptitle('Marginal plot and distributions')

In [ ]:
X_train.shape

* The sample size is $754$ and the dimension of every observation are $4$ , the dimesion $VC$ of a support vector machine is $d+1$ , in this case is $5$ then we can use an heuristics that if $N$ fulfil that $10d_{vc}(\mathcal{H}
)\leq N$ then the model can generalize. In this case $50\leq N$ and for this reason we can give some guarantee of generalization.  

### 4  Provide an explanation how and why the code is working.

To know more about the implementation of the support vector machine, the algorithm of optimization and more, you can check the file *models.py* where are the information and the commented code, in summary we create an object with name *SVM* , now we are going to implement the model.

In [ ]:
svm = SVM()

In [ ]:
X_val[1]

In [ ]:
svm.fit(X_train, y_train,X_val,y_val,learning_rate=0.001,epochs=50)

In [ ]:
svm.plot_loss()


In [ ]:
svm.plot_omega()

### 5 Show some examples to illustrate that the method is working properly.

To do this we create a function with the scaler and the model, in this case we insert the data and see how this works with some examples.

In [ ]:
def svm_model(data):
    X=data.drop('class',axis=1).values
    X=scaler.transform(X)
    return svm.predict(X)

In [ ]:
sum(df_bank_note['class']==0)

In [ ]:
example_data=df_bank_note[750:770]
example_data

In [ ]:
sum(svm_model(example_data)==example_data['class'])

In [ ]:
df_bank_note.drop('class',axis=1)

In [ ]:
datos=scaler.transform(df_bank_note.drop('class',axis=1).values)

In [ ]:
svm.w[0][0]/2

In [ ]:
sum(svm.predict(datos))

# Occupancy 

# Occupancy model 

1) Specify which Machine Learning problem are you solving.

2) Provide a short summary of the features and the labels you are working on.

3. Please answer the following questions: 
    1. Are these datasets linearly separable? 
    2. Are these datasets randomly chosen and 
    3. The sample size is enough to guarantee generalization.

4) Provide an explanation how and why the code is working. You can add comments and/or formal explanations into the notebook.

5) Show some examples to illustrate that the method is working properly.

6) Provide quantitative evidence for generalization using the provided dataset.

https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+

In [ ]:
df_occupancy = pd.read_csv('occupancy_data/datatraining.txt')

In [ ]:
df_occupancy

In [ ]:
# convertir la columna de fechas a un objeto datetime
df_occupancy['date'] = pd.to_datetime(df_occupancy['date'])

# establecer la fecha de referencia
reference_date = pd.to_datetime('2015-02-04')

# convertir fechas a segundos desde la fecha de referencia
df_occupancy['date_seconds'] = (df_occupancy['date'] - reference_date).dt.total_seconds()

In [ ]:
fig , axs = plt.subplots(2,3,figsize=(16,16))

count=0
for i in range(2):
    for j in range(3):
        print(count)
        column=df_occupancy.columns[count]
        axs[i,j].hist(df_occupancy[column],rwidth=0.7)
        print(column)
        axs[i,j].set_title(f'Histogram of {column}')
        count+=1

In [ ]:
X =df_occupancy.drop(['Occupancy','date'],axis=1).values
y= df_occupancy['Occupancy'].replace(0,-1)
y= y.values

# Scale the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df_occupancy.drop(['Occupancy','date'],axis=1)

In [ ]:
X

In [ ]:
# Visualizing data

plt.scatter(df_occupancy['Humidity'],df_occupancy['CO2'],alpha=0.3)

In [ ]:
sns.set_theme(style="dark")
corr = df_occupancy.corr()
mask = np.triu(np.ones_like(corr, dtype=bool)).T
f, ax = plt.subplots(figsize=(6,4))
ax.set_title("Correlation Matrix")
# Generate a custom diverging colormap
# cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
svm_occupancy = SVM()
history = svm_occupancy.fit(X_train, y_train,learning_rate=0.001,epochs=50)

In [ ]:
svm_occupancy.plot_loss()

In [ ]:
X_train.shape